In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('../..')))
cur_path = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(cur_path)
base_path = cur_path + '/data/'
print(base_path)
import numpy as np
import get_dataset
import importlib
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import distributions
from torch.nn.parameter import Parameter
import torch.utils.data as data_utils
from collections import namedtuple
import functools
import random
%load_ext autoreload
%autoreload 2

d:\Microsoft VS Code\PyCodes\RA_Fairness\fair_dummies/data/


In [17]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
os.environ['R_HOME'] = r"D:\R\R-4.3.0" # "/vast/palmer/apps/avx2/software/R/4.3.0-foss-2020b/lib64/R"
os.environ['R_USER'] = r"D:\anaconda3\Lib\site-packages\rpy2"

In [18]:
# importlib.reload(utility_functions)
# importlib.reload(fair_dummies_learning)


from rpy2.robjects.packages import importr
KPC = importr('KPC')
kernlab = importr('kernlab')
import rpy2.robjects
from rpy2.robjects import FloatVector

# specified_density_tr = utility_functions.MAF_density_estimation(Y, A, Y, A)
# specified_density_te = utility_functions.MAF_density_estimation(np.concatenate((Y_cal, Y_test)), np.concatenate((A_cal, A_test)), Y_test, A_test) 
# print("density estimation done")

In [2]:
os.path.abspath(os.path.join('../..'))

'd:\\Microsoft VS Code\\PyCodes\\RA_Fairness\\fair_dummies\\continuous-fairness-master'

In [2]:
from facl.independence.density_estimation.pytorch_kde import kde
from facl.independence.hgr import chi_2_cond, hgr_cond


def chi_squared_kde(X, Y, Z):
    return chi_2_cond(X, Y, Z, kde) #We are going to optimze for EO so we use the conditional version

#You can also use the actual HGR computation function but when one of the variables is binary, the result will be the same
#def hgr_cond(X, Y, Z):
#    return hgr(X, Y, Z, kde)

We download and preprocess the dataset Adult from UCI as in https://github.com/jmikko/fair_ERM

In [3]:
from examples.data_loading import read_dataset

# encoded_data, to_protect, encoded_data_test, to_protect_test = read_dataset(name='adult', fold=1)
# encoded_data.head()

dataset = "adult"
seed = 123
X, A, Y, X_cal, A_cal, Y_cal, X_test, A_test, Y_test = get_dataset.get_train_test_data(base_path, dataset, seed, dim = 3)
X

array([[-0.2134431 ,  0.28120807, -2.70196443, ..., -0.21878026,
        -0.49453079, -5.16482102],
       [-0.2134431 ,  1.05515285,  1.22785281, ..., -0.21878026,
        -1.74376299,  0.26299899],
       [-1.25716323, -1.30130918,  0.17990154, ..., -0.21878026,
         2.58690862,  0.26299899],
       ...,
       [ 1.87399717,  0.12995572, -0.34407409, ..., -0.21878026,
        -0.07812006,  0.26299899],
       [-0.2134431 ,  0.33704024,  0.17990154, ..., -0.21878026,
        -0.49453079,  0.26299899],
       [-0.2134431 ,  0.78559083, -1.39202535, ..., -0.21878026,
        -0.07812006,  0.26299899]])

We  define a very simple neural net 

In [4]:
class NetRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NetRegression, self).__init__()
        size = 64
        self.first = nn.Linear(input_size, size)
        self.last = nn.Linear(size, num_classes)       
    
    def forward(self, x):
        out = F.selu( self.first(x) )
        out = self.last(out)
        return out

A few helper functions to compute performance metrics

In [5]:

def EntropyToProba(entropy): #Only for X Tensor of dimension 2
    return entropy[:,1].exp() / entropy.exp().sum(dim=1)

def calc_accuracy(outputs,Y): #Care outputs are going to be in dimension 2
    max_vals, max_indices = torch.max(outputs,1)
    acc = (max_indices == Y).sum().numpy()/max_indices.size()[0]
    return acc

def results_on_test(model, criterion, encoded_data_test, to_protect_test):
    target = torch.tensor(encoded_data_test['Target'].values.astype(np.longlong)).long()
    to_protect_test = torch.Tensor(to_protect_test)
    data = torch.tensor(encoded_data_test.drop('Target', axis = 1).values.astype(np.float32))
    outputs = model(data).detach()
    loss = criterion(outputs, target)
    p = EntropyToProba(outputs)
    pt = torch.Tensor(to_protect_test)

    ans = {}

    balanced_acc = (calc_accuracy(outputs[to_protect_test==0],target[to_protect_test==0]) +
                    calc_accuracy(outputs[to_protect_test==1],target[to_protect_test==1]))/2

    ans['loss'] = loss.item()
    ans['accuracy'] = calc_accuracy(outputs,target)
    ans['balanced_acc'] = balanced_acc

    f = 0.5
    p1 = (((pt == 1.)*(p>f)).sum().float() / (pt == 1).sum().float())
    p0 = (((pt == 0.)*(p>f)).sum().float() / (pt == 0).sum().float())
    o1 = (((pt == 1.)*(p>f)*(target==1)).sum().float()  / ((pt == 1)*(target==1)).sum().float())
    o2 = (((pt == 0.)*(p>f)*(target==1)).sum().float()  / ((pt == 0)*(target==1)).sum().float())
    di = p1 / p0
    deo = (o1 - o2).abs()
    ans['di'] = di.item()
    ans['deo'] = deo.item()

    return ans


In [6]:
from fair_dummies import fair_dummies_learning
from fair_dummies import utility_functions
importlib.reload(fair_dummies_learning)
importlib.reload(utility_functions)
importlib.reload(get_dataset)

# Hyper Parameters 
input_size = X.shape[1]
num_classes = 2
num_epochs = 200
batch_size = 128
batchRenyi = 256
learning_rate = 1e-2
lambda_renyi = 1


cfg_factory=namedtuple('Config', 'model  batch_size num_epochs lambda_renyi batchRenyi learning_rate input_size num_classes' )


config = cfg_factory(NetRegression, batch_size, num_epochs, lambda_renyi, batchRenyi, learning_rate, input_size, num_classes)

verbose = False

model = config.model(config.input_size, config.num_classes)
        
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=0)

# train_target = torch.tensor(encoded_data['Target'].values.astype(np.longlong)).long()
# train_data = torch.tensor(encoded_data.drop('Target', axis = 1).values.astype(np.float32))
# train_protect = torch.tensor(to_protect).float()

train_target = torch.tensor(Y.astype(np.longlong)).long()
train_data = torch.tensor(X.astype(np.float32))
train_protect = torch.tensor(A).float()
train_tensor = data_utils.TensorDataset(train_data, train_target)

for epoch in range(config.num_epochs):
    train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = config.batch_size, shuffle = True)
    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(x)
        #Select a renyi regularization mini batch and compute the value of the model on it
        frac=config.batchRenyi/train_data.shape[0]
        foo = torch.bernoulli(frac*torch.ones(train_data.shape[0])).byte()
        br = train_data[foo, : ]
        pr = train_protect[foo, :]
        yr = train_target[foo].float()
        ren_outs = model(br)
    
        #Compute the usual loss of the prediction
        loss =  criterion(outputs, y)
        
        #Compte the fairness penalty for positive labels only since we optimize for DEO
        delta =  EntropyToProba(ren_outs)
        #r2 = chi_squared_kde( delta, pr[yr==1.])
        r2 = chi_2_cond(delta, pr, yr, kde, mode = "sum") 
        
        loss += config.lambda_renyi * r2
        
        #In Adam we trust
        loss.backward()
        optimizer.step()
        break
    if verbose:
        print ('Epoch: [%d/%d], Batch: [%d/%d], Loss: %.4f, Accuracy: %.4f, Fairness penalty: %.4f'  % (epoch+1, config.num_epochs, i, len(encoded_data)//batch_size,
                loss.item(),calc_accuracy(outputs,y),
                r2.item()
                    ))
        #print( results_on_test(model, criterion, encoded_data_test, to_protect_test) )

print("Results on test set")
# results_on_test(model, criterion, encoded_data_test, to_protect_test)

Yhat_out_cal = EntropyToProba(model(torch.tensor(X_cal.astype(np.float32)))).detach().flatten().numpy()
Yhat_out_test = EntropyToProba(model(torch.tensor(X_test.astype(np.float32)))).detach().flatten().numpy()
misclassification = sum((Yhat_out_test > 0.5).astype(float) != Y_test) / Y_test.shape[0]
print("misclassification error = " + str(misclassification))

Yhat_out_cal = np.concatenate([(1 - Yhat_out_cal)[:,None], Yhat_out_cal[:,None]], axis = 1)
Yhat_out_test = np.concatenate([(1 - Yhat_out_test)[:,None], Yhat_out_test[:,None]], axis = 1)

outputs = EntropyToProba(model(torch.tensor(X_test.astype(np.float32)))).detach().flatten()
# r2 = np.max(hgr_cond(outputs, torch.tensor(A_test.astype(np.float32)), torch.tensor(Y_test.astype(np.float32)), kde))


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_38036\462570054.py:46: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:28.)
  br = train_data[foo, : ]
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_38036\462570054.py:47: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:28.)
  pr = train_protect[foo, :]
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_38036\462570054.py:48: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:28.)
  yr = train_target[foo].float()
d:\anaconda3\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcomin

Results on test set
misclassification error = 0.20298507462686566


In [34]:

rYhat = rpy2.robjects.r.matrix(FloatVector(Yhat_out_test.T.flatten()), nrow=Yhat_out_test.shape[0], ncol=Yhat_out_test.shape[1])
rZ = rpy2.robjects.r.matrix(FloatVector(A_test.T.flatten()), nrow=A_test.shape[0], ncol=A_test.shape[1]) # rpy2.robjects.r.matrix
rY = rpy2.robjects.r.matrix(FloatVector(Y_test), nrow=A_test.shape[0], ncol=1)

# res = FOCI.codec(Y = rYhat, Z = rZ, X = rY)
# stat = FOCI.codec # KPC.KPCgraph KPC.KPCRKHS FOCI.codec
# res_ = stat(Y = rYhat, X = rY, Z = rZ)[0]
# print(res_)
# res_list = np.zeros(100)
# for i in range(100):
#     At_test = specified_At[i]
#     rZt = rpy2.robjects.r.matrix(FloatVector(At_test.T.flatten()), nrow=A_test.shape[0], ncol=A_test.shape[1])
#     res_list[i] = stat(Y = rYhat, X = rY, Z = rZt)[0]
# p_val = 1.0/(100+1) * (1 + sum(res_list >= res_))
# cur_codec.append(0)
# cur_pval["codec"].append(0)

stat = KPC.KPCgraph # KPC.KPCgraph KPC.KPCRKHS FOCI.codec
res_ = stat(Y = rYhat, X = rY, Z = rZ, Knn = 1)[0]
print(res_)
# p_val = utility_functions.fair_dummies_test_classification(Yhat_out_cal,
#                                                         A_cal,
#                                                         Y_cal,
#                                                         Yhat_out_test,
#                                                         A_test,
#                                                         Y_test,
#                                                         num_reps = 1,
#                                                         num_p_val_rep=1000,
#                                                         reg_func_name="RF",
#                                                         test_type = "MAF_CPT_onA")

0.038778169853435136
